In [1]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNRatingHybridRecommender import ItemKNNRatingHybridRecommender
from KNN.ItemKNNRatingHybridRecommender3 import ItemKNNRatingHybridRecommender3
from KNN.ItemKNNRatingHybridRecommender4 import ItemKNNRatingHybridRecommender4

from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender
from KNN.ItemKNNScoresHybridRecommender3 import ItemKNNScoresHybridRecommender3

from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender4 import ItemKNNSimilarityHybridRecommender4



#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender

In [4]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_all = scipy.sparse.load_npz('./Matrix/URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('./Matrix/ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('./Matrix/URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('./Matrix/URM_test_matrix.npz')

#Load playlistsIDS
data_playlists= pd.read_csv('all/train.csv',low_memory = False)
userID=data_playlists.playlist_id.unique()

CFrecommender = ItemKNNCFRecommender(URM_all)
CFrecommender.fit()

SLIMrecommender = SLIM_BPR_Cython(URM_all)
SLIMrecommender.fit()

CBFrecommender = ItemKNNCBFRecommender(ICM_all,URM_all)
CBFrecommender.fit()

SLIMErecommender = SLIMElasticNetRecommender(URM_all)
SLIMErecommender.fit()

UCFRecommender = UserKNNCFRecommender(URM_all)
UCFRecommender.fit()




In [ ]:
recommender_list = [
    #Random,
    #TopPop,
    #P3alphaRecommender,
    #RP3betaRecommender,
    #ItemKNNCFRecommender,
    #UserKNNCFRecommender,
    #MatrixFactorization_BPR_Cython,
    #MatrixFactorization_FunkSVD_Cython,
    #PureSVDRecommender,
    #SLIM_BPR_Cython,
    SLIMElasticNetRecommender
    ]


from Base.Evaluation.Evaluator import SequentialEvaluator

evaluator = SequentialEvaluator(URM_test, [10], exclude_seen=True)


output_root_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_root_path):
    os.makedirs(output_root_path)


logFile = open(output_root_path + "result_all_algorithms.txt", "a")


for recommender_class in recommender_list:

    try:

        print("Algorithm: {}".format(recommender_class))


        recommender = recommender_class(URM_train)
        recommender.fit()
        result=recommender.evaluateRecommendations(URM_test)
        print("Recommender MAP is= {}".format(result["MAP"]))
        #results_run, results_run_string = evaluator.evaluateRecommender(recommender)
       
        #print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_run_string))
        #logFile.write("Algorithm: {}, results: \n{}\n".format(recommender.__class__, results_run_string))
        #logFile.flush()
 
        
    except Exception as e:
        traceback.print_exc()
        logFile.write("Algorithm: {} - Exception: {}\n".format(recommender_class, str(e)))
        logFile.flush()

Algorithm: <class 'SLIM_ElasticNet.SLIMElasticNetRecommender.SLIMElasticNetRecommender'>


#PER ESEGUIRE SUBMISSION
from createSubmission import create_submission

recommender=UserKNNCFRecommender(URM_all)
recommender.fit()

create_submission(recommender)

import re

results_run_string
m = re.search('MAP: (.+?),', results_run_string)
if m:
    found = m.group(1)
found

#PER CONTROLLARI DOPPIONI RACCOMANDAZIONE
racommandation = CFrecommender.recommend(userToPredict)
racommandation

target_playlist= pd.read_csv('all/target_playlists.csv',low_memory = False)
target_playlistIds=target_playlist.playlist_id.values
target_playlistIds.sort()
songs=data_playlists.loc[data_playlists['playlist_id'].isin(target_playlistIds)]
songs

counter=0
for x in target_playlistIds:
        raccomandate=racommandation[counter]
        dfsongs = songs.loc[songs['playlist_id'] == x]
        for row in dfsongs.itertuples():
            if row.track_id in raccomandate:
                print("ERRORRRR")
                print(x)
                print(row.track_id)
        counter=counter+1



In [ ]:
from createSubmission import create_submission


SuperSlimSimilarity=ItemKNNSimilarityHybridRecommender(URM_all,SLIMrecommender,SLIMErecommender,"l2")
SuperSlimSimilarity.fit(alpha=0.5, topK=600)

recommender=ItemKNNSimilarityHybridRecommender3(URM_all,SuperSlimSimilarity, CFrecommender,CBFrecommender,"l2")
recommender.fit(topK=600,alpha=0.6,beta=0.2,gamma=0.2)

create_submission(recommender)